In [26]:
############### Importing All Libraries #####################
import os, glob
from datetime import date
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

In [27]:
############### Creating SparkSession and Creating EmptyRDD #####################
spark = SparkSession.builder.appName('nyc_processed_zone').getOrCreate()
emptyRDD = spark.sparkContext.emptyRDD()

In [28]:
############### getting current date, month and day #####################
today = date.today()
year = str("year="+str(today.strftime("%Y")))
month = str("month="+str((today.strftime("%b"))[:3]))
day = str("day="+str((today.strftime("%d"))))

In [29]:
############### Get list of file's and folder's #####################
glob.glob('/opt/notebooks/nyc/processed_zone/trips/2021/11/*')

[]

In [30]:
# new_df = spark.read.parquet('/opt/notebooks/nyc/processed_zone/trips/2021/11/25/*/*')

In [32]:
# spark.read.parquet("/opt/notebooks/nyc/processed_zone/trips/{0}/{1}/{2}".format(year,month,day))

In [41]:
############### checking the folder or file exists or not #####################
if (os.path.exists("/opt/notebooks/nyc/processed_zone/trips/{0}".format(year))):
    print(year)
    if (os.path.exists("/opt/notebooks/nyc/processed_zone/trips/{0}/{1}".format(year,month))):
        print(month)
        if (os.path.exists("/opt/notebooks/nyc/processed_zone/trips/{0}/{1}/{2}".format(year,month,day))):
            print(day)
            ############### reading data from new path #####################
            new_path = "/opt/notebooks/nyc/processed_zone/trips/{0}/{1}/{2}".format(year,month,day)
            #//////this is manual part for now
            new_df = spark.read.parquet(str("/opt/notebooks/nyc/processed_zone/trips/{0}/{1}/{2}".format(year,month,day)+"*/*"))
            ############### checking the folder or file exists or not #####################
            if (os.path.exists("/opt/notebooks/nyc/Hive_Processed_Data")):
                print("Hive_Processed_Data exists")
                if (len(glob.glob("/opt/notebooks/nyc/Hive_Processed_Data/*"))):
                    print("old data parquet file exists")
            
                    ############### reading data from old path #####################
                    old_path = "/opt/notebooks/nyc/Hive_Processed_Data/*"
                    old_df = spark.read.parquet(old_path)

                    ############### union of (new + old) Data #####################
                    union_df = new_df.unionAll(old_df)

                    ############### Adding new columns row_number #####################
                    df_with_columns = union_df.withColumn("row_number", row_number().over(Window.partitionBy(union_df['trip_id']).orderBy('trip_id')))


                    ############### droping the row_numbers col #####################
                    df_without_col = df_with_columns.where(df_with_columns.row_number == 1).drop("row_number")


                    ############### Saving data agin to the same path #####################
                    df_without_col.repartition(1).write.format("parquet").mode("append").save("/opt/notebooks/nyc/Hive_Processed_Data/final_hive.parquet")
                else:
                    print('file not exists inside Hive_Processed_Data folder')
                    new_df.repartition(1).write.format("parquet").save("/opt/notebooks/nyc/Hive_Processed_Data/final_hive.parquet")
            else:
                print('Hive_Processed_Data does not exists')
     

year=2022
month=Apr
day=28
Hive_Processed_Data exists
old data parquet file exists


In [39]:
# new_path = "/opt/notebooks/nyc/processed_zone/trips/{0}/{1}/{2}".format(2021,11,25)

In [40]:
# new_path